# Parte 2:  Sistema RAG - Ingesta y Adquisición de Conocimiento
*   **Autor:** Carolina Torres Zapata
*   **Fecha:** 2025-11-24
*   **Contexto:**
La base de un sistema RAG (Retrieval-Augmented Generation) es la calidad de sus datos. En este notebook implementamos la primera fase del pipeline ETL para LLMs: la adquisición de datos no estructurados.
*  **Objetivo:**
     1.  Conectarse a una fuente externa (Documentación oficial de Microsoft Learn).
     2.  Extraer el contenido HTML y aplicar una limpieza estructural (eliminar ruido HTML).
     3.  Persistir la información cruda en la **Capa Bronce** utilizando **Unity Catalog Volumes**, asegurando la trazabilidad del dato fuente antes del procesamiento (chunking).

## 1. Importar Librerías

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
import requests
from bs4 import BeautifulSoup

## 2. Extracción y Limpieza (Web Scraping)
Simulamos la ingesta de un documento técnico. Para garantizar la calidad de los embeddings posteriores, aplicamos una estrategia de limpieza:

1.  **Request Robusto:** Verificación de estado HTTP (200 OK) y forzado de encoding UTF-8 para manejo correcto de tildes/eñes.
2.  **Eliminación de Ruido:** Removemos etiquetas HTML irrelevantes para el contenido (`<script>`, `<style>`, `<nav>`, `<footer>`) que solo añadirían tokens basura al contexto del LLM.
3.  **Extracción Selectiva:** Solo conservamos encabezados (`h1`-`h3`), párrafos (`p`) y listas (`li`) dentro del contenedor principal.


In [0]:

# URL oficial de Microsoft Learn
url = "https://learn.microsoft.com/es-es/azure/databricks/introduction/"

# 1. Descargar HTML
response = requests.get(url)
response.raise_for_status()


# Codificación a UTF-8 para que lea bien las tildes y ñ
response.encoding = 'utf-8' 

# 2. Parsear HTML
soup = BeautifulSoup(response.text, "html.parser")

# 3. Limpieza de elementos irrelevantes
for tag in soup(["script", "style", "nav", "footer", "header", "aside", "meta", "link"]): 
    tag.decompose() 

# 4. Extraer contenido principal
main = soup.find(id="main") or soup 

# 5. Extraer texto limpio
texts = [] 
for tag in main.find_all(["h1", "h2", "h3", "p", "li"]): 
    # get_text strip=True quita espacios extra
    # replace('\xa0', ' ') quita los "espacios de no separación" que ensucian el texto
    content = tag.get_text(" ", strip=True).replace('\xa0', ' ')
    if content: 
        texts.append(content) 

# Unir todo
full_text = "\n\n".join(texts)

# Verificamos
print(full_text[:1000]) # Imprimimos los primeros 500 caracteres para validar

Nota:

El acceso a esta página requiere autorización. Puede intentar iniciar sesión o cambiar directorios .

El acceso a esta página requiere autorización. Puede intentar cambiar los directorios .

¿Qué es Azure Databricks?

Azure Databricks es una plataforma de análisis unificada y abierta para crear, implementar, compartir y mantener soluciones de datos, análisis e IA de nivel empresarial a escala. Databricks Data Intelligence Platform se integra con el almacenamiento en la nube y la seguridad de su cuenta en la nube, y administra e implementa la infraestructura en la nube para usted.

Azure Databricks usa inteligencia artificial generativa con el almacén de lago de datos para comprender la semántica única de los datos. A continuación, optimiza automáticamente el rendimiento y administra la infraestructura para adaptarla a las necesidades de su empresa.

El procesamiento de lenguaje natural aprende el idioma de su empresa, por lo que puede buscar y detectar datos haciendo una pregunt

## 3. Persistencia en Capa Bronce (Unity Catalog Volumes)
En lugar de procesar y "chunkear" inmediatamente en memoria, guardamos el texto limpio como un archivo físico en un **Volumen de Unity Catalog**.

**Por qué hacemos esto (Soporte Operacional):**
*   **Auditabilidad:** Tenemos una copia fiel de lo que se descargó en ese momento.
*   **Desacoplamiento:** Si cambiamos la estrategia de *chunking* (ej. de 500 a 1000 caracteres), no necesitamos volver a hacer scraping de la web, solo releemos este archivo Bronce.

In [0]:
# Ruta donde guardar el archivo en la capa Bronze
output_path = "/Volumes/dev/bronce/azure_databricks_docs/azure_databricks_intro.txt"

# Guardar el texto como archivo .txt
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_text)

print("Archivo guardado correctamente en:", output_path)


Archivo guardado correctamente en: /Volumes/dev/bronce/azure_databricks_docs/azure_databricks_intro.txt
